# 強化学習とABCクラス

* 参考：[A General Framework for Sample-Efficient Function Approximation in Reinforcement Learning](https://arxiv.org/abs/2209.15634)

強化学習の複雑度として，Bellman rankやBilinear class，Bellman Eluder dimensionなど様々なものが開発されてきました．
[Decision Estimation Coefficient](RL_General_DEC.ipynb)はかなり広範なクラスをカバーできますが，DEC用のアルゴリズムはMinimaxサンプル効率を達成しません．
ほぼすべてのクラスを含み，かつタイトなサンプル効率になるようなクラスは作れないでしょうか？

![ABC](figs/ABC.png)


**準備**

* 仮説クラス：$\mathcal{F}:=\mathcal{F}_1 \times \ldots \times \mathcal{F}_H$
    * 仮説に対する価値関数：$\left\{Q_f, V_f\right\}=\left\{Q_{h, f}, V_{h, f}\right\}_{h \in[H]}$
    * 仮説に対する方策：$\pi_f(s)=\arg \max _\pi \mathbb{E}_{a \sim \pi}\left[Q_{h, f}(s, a)\right]$
  
次を仮定します

---

**Realizability**

次を満たす仮説$f^* \in \mathcal{F}$が存在する．

$$Q_h^*(s, a)=Q_{h, f^*}(s, a)$$

---

次のCovering numberを導入します．

---

**$\epsilon$-covering number**

任意の$\epsilon > 0$と仮説クラス$\mathcal{F}$に対して，$N_\mathcal{F}(\epsilon)$が次を満たすとき，$\epsilon$-covering numberと呼びます．

任意の$f \in \mathcal{F}$について，$\rho\left(f, f^{\prime}\right) \leq \epsilon$であるような$f'\in \mathcal{F}_\epsilon$が存在するとき，$\mathcal{F}_\epsilon$の要素数のこと．

---

そして，Eluder dimensionを一般化するために，Functional Eluder Dimensionを導入します．

---

**Functional Eluder Dimension**

仮説クラス$\mathcal{F}$と$\mathcal{F} \times \mathcal{F}$上の関数$G$に対して，次を満たす$\operatorname{dim}_{F E}(\mathcal{F}, G, \epsilon)$を，Functional Eluder dimensionと呼びます．

任意の$2 \leq t \leq n$と，なんらかの$\epsilon' \geq \epsilon$について，$\sqrt{\sum_{i=1}^{t-1}\left(G\left(g, f_i\right)\right)^2} \leq \epsilon^{\prime}$ while $\left|G\left(g, f_t\right)\right|>\epsilon^{\prime}$を満たすような$g \in \mathcal{F}$が存在するような最長の系列$f_1, \ldots, f_n \in \mathcal{F}$の長さを$\operatorname{dim}_{F E}(\mathcal{F}, G, \epsilon)$とする．

---

これは明らかにEluder dimensionを一般化してますね．また，例えば$G_h(g, f):=\mathbb{E}_{\pi_{h, f}}\left(Q_{h, g}-\mathcal{T}_h Q_{g, h+1}\right)$とすれば，これはBellman Eluder dimensionと同じです．


## ABCクラス

準備：
* MDP : $M$
* $s_1, a_1, \dots, S_H$
* 仮説クラス：$\mathcal{F} \subset \mathcal{G}$
* $\mathcal{V} = \left\{v\left(s, a, s^{\prime}\right): \mathcal{S} \times \mathcal{A} \times \mathcal{S} \rightarrow \mathbb{R}\right\}$
* $\ell=\left\{\ell_{h, f^{\prime}}\right\}_{h \in[H], f^{\prime} \in \mathcal{F}}$ は $\mathbb{R}^{d_s}$次元関数．
    * $o_h:=\left(s_h, a_h, s_{h+1}\right) \in \mathcal{S} \times \mathcal{A} \times \mathcal{S}, f \in \mathcal{F}, g \in \mathcal{G}$ and $v \in \mathcal{V}$ を入力されると，Bellman 誤差のSurrogateを返します．
    * また，$\left\|\ell_{h, f^{\prime}}\left(\cdot, f_{h+1}, g_h, v\right)\right\|_2 \leq R$ for $\forall h \in[H], f^{\prime}, f \in \mathcal{F}, g \in \mathcal{G}, v \in \mathcal{V}$を仮定します．
* $G_{h, f^*}(f, g)$を$\mathcal{F}\times \mathcal{F}$上の関数とし，$f, g\in \mathcal{F}$同士の関係を表す関数とします（後で具体例が出ます）

このとき，次のAdmissible Bellman Characterizationを導入します．

---

**Admissible Bellman Characterization**

MDP $M$，realizabilityを満たす仮説クラス$\mathcal{F} \subset \mathcal{G}$，そして，推定関数$\ell_{h, f'}: (\mathcal{S} \times \mathcal{A} \times \mathcal{S}) \times \mathcal{F} \times \mathcal{G} \times \mathcal{V} \rightarrow \mathbb{R}^{\mathbb{d}_s}$，探索方策$\pi_{\text{op}}$そして定数$\kappa (0, 1]$に対して，次を満たすGを，$(M, \mathcal{F}, \mathcal{G}, \ell)$のadmissible Bellman characterizationと呼びます：

1. Dominating Average Estimation Function: 任意の$f, g\in \mathcal{F}$について，

$$
\max _{v \in \mathcal{V}} \mathbb{E}_{s_h \sim \pi_g, a_h \sim \pi_{\mathrm{op}}}\left\|\mathbb{E}_{s_{h+1}}\left[\ell_{h, g}\left(o_h, f_{h+1}, f_h, v\right) \mid s_h, a_h\right]\right\|_2^2 \geq\left(G_{h, f^*}(f, g)\right)^2
$$

2. Bellman Dominance: 任意の$(h, f) \in [H] \times \mathcal{F}$について，

$$
\kappa \cdot\left|\mathbb{E}_{s_h, a_h \sim \pi_f}\left[Q_{h, f}\left(s_h, a_h\right)-r\left(s_h, a_h\right)-V_{h+1, f}\left(s_{h+1}\right)\right]\right| \leq\left|G_{h, f *}(f, f)\right|
$$

また，
$(M, \mathcal{F}, \mathcal{G}, \ell)$のことを，ABCクラスと呼びます．

---

**他のクラスとの違い**

$$
G_{h, f^*}(f, g)=\left\langle W_h(g)-W_h\left(f^*\right), X_h(f)\right\rangle
$$
とすると，明らかにABCクラスはBilinearクラスになります．また，
$$
\ell_{h, f^{\prime}}\left(o_h, f_{h+1}, g_h, v\right):=Q_{h, g}\left(s_h, a_h\right)-r_h-V_{h+1, f}\left(s_{h+1}\right) .
$$
とすれば，明らかにlow Bellman Eluder dimensionと同じです．


---

**Decomposable Estimation Function**

推定関数
$\ell_{h, f'}: (\mathcal{S} \times \mathcal{A} \times \mathcal{S}) \times \mathcal{F} \times \mathcal{G} \times \mathcal{V} \rightarrow \mathbb{R}^{\mathbb{d}_s}$に対して，
次の構造を考えます．

1. （**Decomposability**）: 任意の$f \in \mathcal{F},\left(h, f^{\prime}, g, v\right) \in[H] \times \mathcal{F} \times \mathcal{G} \times \mathcal{V}$と任意の$o_h$について，次を満たす写像$\mathcal{T}(\cdot): \mathcal{F}\to \mathcal{G}$が存在する：
$$
\ell_{h, f^{\prime}}\left(o_h, f_{h+1}, g_h, v\right)-\mathbb{E}_{s_{h+1}}\left[\ell_{h, f^{\prime}}\left(o_h, f_{h+1}, g_h, v\right) \mid s_h, a_h\right]=\ell_{h, f^{\prime}}\left(o_h, f_{h+1}, \mathcal{T}(f)_h, v\right)
$$
また，$f=f^*$なら，$\mathcal{T}(f)=f^*$であるとする．
（Decomposabilityは推定関数がベルマン誤差っぽい挙動をしていることを表してます．）

例えば
$$
\ell_{h, f^{\prime}}\left(o_h, f_{h+1}, g_h, v\right):=Q_{h, g}\left(s_h, a_h\right)-r\left(s_h, a_h\right)-V_{h+1, f}\left(s_{h+1}\right)
$$
の場合は，
$$
\begin{aligned}
& {\left[Q_{h, g}\left(s_h, a_h\right)-r\left(s_h, a_h\right)-V_{h+1, f}\left(s_{h+1}\right)\right]-\left[Q_{h, g}\left(s_h, a_h\right)-\left(\mathcal{T}_h V_{h+1}\right)\left(s_h, a_h\right)\right]} \\
& =\left(\mathcal{T}_h V_{h+1}\right)\left(s_h, a_h\right)-r\left(s_h, a_h\right)-V_{h+1, f}\left(s_{h+1}\right) .
\end{aligned}
$$

と同じです．

また，特に$f=g=f^*$なら，
$$
\ell_{h, f^{\prime}}\left(o_h, f^*_{h+1}, f^*_h, v\right)-\mathbb{E}_{s_{h+1}}\left[\ell_{h, f^{\prime}}\left(o_h, f^*_{h+1}, f^*_h, v\right) \mid s_h, a_h\right]=\ell_{h, f^{\prime}}\left(o_h, f^*_{h+1}, f^*_h, v\right)
$$
なので，
$$
\mathbb{E}_{s_{h+1}}\left[\ell_{h, f^{\prime}}\left(o_h, f^*_{h+1}, f^*_h, v\right) \mid s_h, a_h\right]=0
$$
です．

2. (**Global Discriminator Optimality**): 任意の$f \in \mathcal{F}$に対して，次を満たす大域的極大値$v_h^*(f) \in \mathcal{V}$が存在する．任意の$\left(h, f^{\prime}, g, v\right) \in[H] \times \mathcal{F} \times \mathcal{G} \times \mathcal{V}$と$o_h$について，
$$
\left\|\mathbb{E}_{s_{h+1}}\left[\ell_{h, f^{\prime}}\left(o_h, f_{h+1}, f_h, v_h^*(f)\right) \mid s_h, a_h\right]\right\|_2 \geq\left\|\mathbb{E}_{s_{h+1}}\left[\ell_{h, f^{\prime}}\left(o_h, f_{h+1}, f_h, v\right) \mid s_h, a_h\right]\right\|_2
$$
（任意の$(s_h, a_h)$に対して，discriminatorの大域的極大値が存在することが重要っぽいです）


最後に，Estimation functionにリプシッツ連続性を導入します．

---

**リプシッツ連続性**

任意の
$\left(h, f^{\prime}, f, g, v\right) \in [H] \times \mathcal{F} \times \mathcal{F} \times \mathcal{G} \times \mathcal{V},\left(\tilde{f}, \tilde{g}, \tilde{v}, \tilde{f}^{\prime}\right) \in \mathcal{F} \times \mathcal{G} \times \mathcal{V} \times \mathcal{F}$そして任意の$o_h$について，

$$
\begin{aligned}
& \left\|\ell_{h, f^{\prime}}(\cdot, f, g, v)-\ell_{h, f^{\prime}}(\cdot, \tilde{f}, g, v)\right\|_{\infty} \leq L \rho(f, \tilde{f}), \quad\left\|\ell_{h, f^{\prime}}(\cdot, f, g, v)-\ell_{h, f^{\prime}}(\cdot, f, \tilde{g}, v)\right\|_{\infty} \leq L \rho(g, \tilde{g}), \\
& \left\|\ell_{h, f^{\prime}}(\cdot, f, g, v)-\ell_{h, f^{\prime}}(\cdot, f, g, \widetilde{v})\right\|_{\infty} \leq L\|v-\widetilde{v}\|_{\infty}, \quad\left\|\ell_{h, f^{\prime}}(\cdot, f, g, v)-\ell_{h, \tilde{f}^{\prime}}(\cdot, f, g, v)\right\|_{\infty} \leq L \rho\left(f^{\prime}, \tilde{f}^{\prime}\right) .
\end{aligned}
$$

を満たすような$L > 0$が存在する．

---

また，
$\mathcal{L}=\left\{\ell_{h, f^{\prime}}(\cdot, f, g, v):\left(h, f^{\prime}, f, g, v\right) \in[H] \times
\mathcal{F} \times \mathcal{F} \times \mathcal{G} \times \mathcal{V}\right\} $
を，誘導される推定関数のクラスとします．
ちなみに上のリプシッツ関数の仮定のもとでは，
$N_{\mathcal{L}}(\epsilon) \leq N_{\mathcal{F}}^2\left(\frac{\epsilon}{4 L}\right) N_{\mathcal{G}}\left(\frac{\epsilon}{4 L}\right) N_{\mathcal{V}}\left(\frac{\epsilon}{4 L}\right)$
が成立します．ここで，
$N_{\mathcal{F}}(\epsilon), N_{\mathcal{G}}(\epsilon), N_{\mathcal{F}}(\epsilon)$
はそれぞれ対応する仮説クラスの$\epsilon$-covering numberです．


## Operaアルゴリズム

ABCクラスに対して，次のOPERAアルゴリズムを考えます．

![OPERA](figs/OPERA.png)


このアルゴリズムについて，次のリグレットバウンドが成立します．

---

MDP $M$と仮説クラス$\mathcal{F}, \mathcal{G}$，Decomposable Estimation Function $\ell$，そして admissible Bellman characterization $G$を考えます．また，$(M, \mathcal{F}, \mathcal{G}, \ell, G)$がABCクラスであり，Low functional eluder dimensionであるとします．このとき，任意の$\delta \in (0, 1)$について，
$\beta=\mathcal{O}\left(\log \left(T H N_{\mathcal{L}}(1 / T) / \delta\right)\right)$とします．
このとき，on-policyな場合，つまり，$\pi_{\mathrm{op}}=\pi_{\mathrm{est}}=\pi^t$のとき，リグレットは

$$
\operatorname{Regret}(T)=\mathcal{O}\left(\frac{H}{\kappa} \sqrt{T \cdot \operatorname{dim}_{\mathrm{FE}}(\mathcal{F}, G, \sqrt{1 / T}) \cdot \beta}\right)
$$

で抑えられます．


**証明**

### ステップ１：$f^*$の実行可能性

まず，$f^*$が実行可能解であることを示します．

---

アルゴリズム１において，$\rho > 0$と$\delta > 0$に対して，
$\beta = c\left(\log \left(T H \mathcal{N}_{\mathcal{L}}(\rho) / \delta\right)+T\rho \right)$
を十分大きな$c$で設定するとします．
ここで$\rho$は$\mathcal{L}$の$\epsilon$-covering numberかな？
このとき，確率$1-\delta$以上で，

$$
\max _{v \in \mathcal{V}}\left\{\sum_{i=1}^{t-1}\left\|\ell_{h, f_h^i}\left(o_h^i, f_{h+1}^*, f_h^*, v\right)\right\|_2^2-\inf _{g_h \in \mathcal{G}_h} \sum_{i=1}^{t-1}\left\|\ell_{h, f_h^i}\left(o_h^i, f_{h+1}^*, g_h, v\right)\right\|_2^2\right\} \leq \mathcal{O}(\beta)
$$

が任意の$t\in T$で成立します．

**証明**

まずは固定された$f \in \mathcal{F}$と$v \in \mathcal{V}$を考えます．

データセット
$\mathcal{D}_h=\left\{r_h^t, s_h^t, a_h^t, s_{h+1}^t\right\}_{t=1,2, \ldots T}$
について，
$$
X_i(h, f, v):=\left\|\ell_{h, f^i}\left(o_h^i, f_{h+1}^*, f_h, v\right)\right\|_2^2-\left\|\ell_{h, f^i}\left(o_h^i, f_{h+1}^*, f_h^*, v\right)\right\|_2^2 .
$$
なる確率変数を導入します．
このとき，$\left|X_t(h, f, v)\right| \leq R^2$です．
また，$s_{h+1}$についての期待値を考えると，$\|a\|^2-\|b\|^2=\langle a-b, a+b\rangle$から，

$$
\begin{aligned}
& \mathbb{E}_{s_{h+1}}\left[X_i(h, f, v) \mid s_h, a_h\right] \\
& =\mathbb{E}_{s_{h+1}}\left[\left\|\ell_{h, f^i}\left(o_h^i, f^*_{h+1}, f_h, v\right)\right\|_2^2-\left\|\ell_{h, f^i}\left(o_h^i, f^*_{h+1}, f^*_h, v\right)\right\|_2^2 \mid s_h, a_h\right]\\
& = \mathbb{E}_{s_{h+1}}\left[\left\langle\ell_{h, f^i}\left(o_h^i, f_{h+1}^*, f_h, v\right)-\ell_{h, f^{\prime}}\left(o_h^i, f^*_{h+1}, f^*_h, v\right), \ell_{h, f^i}\left(o_h^i, f^*_{h+1}, f_h, v\right)+\ell_{h, f^{\prime}}\left(o_h^i, f^*_{h+1}, f^*_h, v\right)\right\rangle  \mid s_h, a_h\right]\\
\end{aligned}
$$
が成り立ちます（元論文は普通に表記ミスってる）．ここで，Decomposabilityから
$$
\ell_{h, f^{\prime}}\left(o_h, f^*_{h+1}, f_h, v\right)-\ell_{h, f^{\prime}}\left(o_h, f^*_{h+1}, f^*_h, v\right)
=\mathbb{E}_{s_{h+1}}\left[\ell_{h, f^{\prime}}\left(o_h, f^*_{h+1}, f_h, v\right) \mid s_h, a_h\right]
$$

なので，
$$
\begin{aligned}
& \mathbb{E}_{s_{h+1}}\left[X_i(h, f, v) \mid s_h, a_h\right] \\
& = \mathbb{E}_{s_{h+1}}\left[\left\langle\ell_{h, f^i}\left(o_h^i, f_{h+1}^*, f_h, v\right)-\ell_{h, f^{\prime}}\left(o_h^i, f^*_{h+1}, f^*_h, v\right), \ell_{h, f^i}\left(o_h^i, f^*_{h+1}, f_h, v\right)+\ell_{h, f^{\prime}}\left(o_h^i, f^*_{h+1}, f^*_h, v\right)\right\rangle  \mid s_h, a_h\right]\\
& = \mathbb{E}_{s_{h+1}}\left[\left\langle \mathbb{E}_{s_{h+1}}\left[\ell_{h, f^{\prime}}\left(o_h, f^*_{h+1}, f_h, v\right) \mid s_h, a_h\right] , \ell_{h, f^i}\left(o_h^i, f^*_{h+1}, f_h, v\right)+\ell_{h, f^{\prime}}\left(o_h^i, f^*_{h+1}, f^*_h, v\right)\right\rangle  \mid s_h, a_h\right]\\
& = \left\langle \mathbb{E}_{s_{h+1}}\left[\ell_{h, f^{\prime}}\left(o_h, f^*_{h+1}, f_h, v\right) \mid s_h, a_h\right] , \mathbb{E}_{s_{h+1}}\left[\ell_{h, f^i}\left(o_h^i, f^*_{h+1}, f_h, v\right)+\ell_{h, f^{\prime}}\left(o_h^i, f^*_{h+1}, f^*_h, v\right) \mid s_h, a_h\right]\right\rangle \\
& = \left\langle \mathbb{E}_{s_{h+1}}\left[\ell_{h, f^{\prime}}\left(o_h, f^*_{h+1}, f_h, v\right) \mid s_h, a_h\right] , \mathbb{E}_{s_{h+1}}\left[\ell_{h, f^i}\left(o_h^i, f^*_{h+1}, f_h, v\right)  \mid s_h, a_h\right]\right\rangle\\
&=\left\|\mathbb{E}_{s_{h+1}}\left[\ell_{h, f^i}\left(o_h^i, f_{h+1}, f_h, v\right) \mid s_h, a_h\right]\right\|_2^2 .
\end{aligned}
$$

一方で，
$$
\begin{aligned}
\mathbb{E}_{s_{h+1}}\left[\left(X_i(h, f, v)\right)^2 \mid s_h, a_h\right] 
\leq & \mathbb{E}_{s_{h+1}}\left[\| \ell_{h, f^i}\left(o_h^i, f^*_{h+1}, f_h, v\right)-\ell_{h, f^i}\left(o_h^i, f^*_{h+1}, f^*_h, v\right)\|_2^2 \right.\\
& \left.\cdot\left\|\ell_{h, f^{\prime}}\left(o_h^i, f^*_{h+1}, f_h, v\right)+\ell_{h, f^{\prime}}\left(o_h^i, f^*_{h+1}, f^*_h, v\right)\right\|_2^2 \mid s_h, a_h\right] \\
\leq & 4\left\|\mathbb{E}_{s_{h+1}}\left[\ell_{h, f^i}\left(o_h^i, f_{h+1}, f_h, v\right) \mid s_h, a_h\right]\right\|_2^2 R^2 \\
\leq & 4 R^2 \mathbb{E}_{s_{h+1}}\left[X_i(h, f, v) \mid s_h, a_h\right] .
\end{aligned}
$$

も成り立ちます．２行目の変形にはDecomposabilityと$\ell$の上界を使えばいけます．３行目ではひとつ上の等式変形を使いました．


ここで，Freedmanの不等式を導入します：

---

**Freedmanの不等式**

$X_1, X_2, \dots, X_T$を実数の確率変数の列とします．
任意の$t\in \{1, 2, \dots, T\}$において，$X_t \leq R$および$\mathbb{E}\left[X_t \mid X_1, \ldots, X_{t-1}\right]=0$とします．
このとき，任意の$\delta \in (0, 1)$ と$\eta \in [0, 1/R]$について，確率$1-\delta$以上で，

$$
\sum_{t=1}^T X_t 
\leq(e-2) \eta \sum_{t=1}^T \mathbb{E}\left[X_t^2 \mid X_1, \dots, X_{t-1} \right]+\frac{\log \left(\delta^{-1}\right)}{\eta}
$$

証明は[Taming the Monster: A Fast and Simple Algorithm for Contextual Bandits](https://arxiv.org/abs/1402.0555)を参考にしてください．

---


ここで，
$Z_t=\mathbb{E}_{s_{h+1}}\left[X_t(h, f, v) \mid s_h, a_h\right] - X_t(h, f, v)$ とすると，$\mathbb{E}[Z_t\mid Z_1, \dots, Z_{t-1}]=0$であり，$\left|Z_t\right| \leq 2 R^2$です．

Freedmanの不等式を使うと，

$$
\begin{aligned}
& \sum_{i=1}^t \mathbb{E}_{s_{h+1}}\left[X_i(h, f, v) \mid s_h, a_h\right] - \sum_{i=1}^t X_i(h, f, v)\\
\leq & (e-2) \eta \sum_{i=1}^t \mathbb{E}_{s_{h+1}}\left[\left(X_i(h, f, v) - \mathbb{E}_{s_{h+1}}\left[X_i(h, f, v) \mid s_h, a_h\right]\right)^2\mid s_h, a_h\right] +\frac{\log \left(\delta^{-1}\right)}{\eta}\\
\leq & (e-2) \eta \sum_{i=1}^t \mathbb{E}_{s_{h+1}}\left[\left(X_i(h, f, v)\right)^2\mid s_h, a_h\right] +\frac{\log \left(\delta^{-1}\right)}{\eta}\\
\leq & (e-2) 4 R^2 \eta \sum_{i=1}^t \mathbb{E}_{s_{h+1}}\left[X_i(h, f, v) \mid s_h, a_h\right] +\frac{\log \left(\delta^{-1}\right)}{\eta}
\end{aligned}
$$

が成立します．
ここで２つ目は$\mathbb{V}[X]= \mathbb{E}[X^2] - (\mathbb{E}[X])^2 \leq \mathbb{E}[X^2]$を使っており，３つ目は上でやった不等式です．
 
$\eta=\frac{1}{4(e-2) R^2}$とすると，
$$
\begin{aligned}
- \sum_{i=1}^t X_i(h, f, v)
\leq 4(e-2) R^2\log \left(\delta^{-1}\right) = \mathcal{O}(R^2 \log(\delta^{-1}))
\end{aligned}
$$
が確率$1-\delta$以上で成立します．

続いて，この結果を任意の$f$と$v$に拡張しましょう．

まず，リプシッツ連続性の仮定より，$\mathcal{L}$のcovering numberは$\mathcal{F}, \mathcal{G}, \mathcal{V}$のcovering numberで抑えることができます．
よって，任意の$\ell_{h, f^{\prime}}(\cdot, f, g, v)$と$\rho > 0$について，$\tilde{\rho}$と$\left(\tilde{f}^{\prime}, \tilde{f}, \widetilde{g}, \widetilde{v}\right) \in \mathcal{F}_{\widetilde{\rho}} \times \mathcal{F}_{\widetilde{\rho}} \times \mathcal{G}_{\widetilde{\rho}} \times \mathcal{V}_{\widetilde{\rho}}$が存在し，

$$
\left\|\ell_{h, \widetilde{f}^{\prime}}(\cdot, \widetilde{f}, \widetilde{g}, \widetilde{v})-\ell_{h, f^{\prime}}(\cdot, f, g, v)\right\|_{\infty}
\leq \rho
$$
が成立します．ここで，$\mathcal{F}_{\widetilde{\rho}}, \mathcal{G}_{\widetilde{\rho}}, \mathcal{V}_{\widetilde{\rho}}$は$\mathcal{F}, \mathcal{G}, \mathcal{V}$の$\tilde{\rho}$-coverです．
つまり，$\left(\widetilde{f^{\prime}}, \widetilde{f}, \widetilde{g}, \widetilde{v}\right) \in \mathcal{L}_\rho$が成立してます．

$$
\tilde{X}_i(h, \tilde{f}, \widetilde{v}):=\left\|\ell_{h, \tilde{f}^i}\left(o_h^i, {f}^*_{h+1}, \tilde{f}_h, \widetilde{v}\right)\right\|_2^2-\left\|\ell_{h, \tilde{f}^i}\left(o_h^i, {f}^*_{h+1}, {f}^*_h, \widetilde{v}\right)\right\|_2^2
$$
および
$$
\iota=\log \left(\frac{H T \mathcal{N}_{\mathcal{L}}(\rho)}{\delta}\right)
$$
とします．
$\mathcal{L}_\rho$上のUnion boundを取ると，

$$
\begin{aligned}
- \sum_{i=1}^t \tilde{X}_i(h, \tilde{f}, \tilde{v})
\leq \mathcal{O}(R^2 \iota)
\end{aligned}
$$
が任意の$(\tilde{f}^i, \tilde{f}, \tilde{v}) \in \mathcal{L}_\rho$で成立します．

さて，$X_i(h, f, v)$について，covering numberを使えば（多分$|X_i - \tilde{X}_i| \leq \rho$になるように選べるはず）

$$
\begin{aligned}
-\sum_{i=1}^{t} X_i\left(h, {f}, v\right)
=&-\sum_{i=1}^{t} 
\widetilde{X}_i\left(h, \widetilde{f}, \widetilde{v}\right)
+ X_i\left(h, {f}, v\right)
-\widetilde{X}_i\left(h, \widetilde{f}, \widetilde{v}\right)\\
\leq&-\sum_{i=1}^{t} 
\widetilde{X}_i\left(h, \widetilde{f}, \widetilde{v}\right)
+ |X_i\left(h, {f}, v\right) -\widetilde{X}_i\left(h, \widetilde{f}, \widetilde{v}\right)|\\
\leq &\mathcal{O}(R^2 \iota + Rt\rho)
\end{aligned}
$$

が任意の$f, v$で成立する．
これで証明終わりです．

## ステップ２（リグレットの分解）

ステップ１よりOptimismが成立し，

$$V_1^*\left(s_1\right)-V_1^{\pi^t}\left(s_1\right) \leq V_{1, f^t}\left(s_1\right)-V_1^{\pi^t}\left(s_1\right)$$

が成立します．Performance difference Lemma を使えば，

$$
V_1^*\left(s_1\right)-V_1^{\pi^t}\left(s_1\right) \leq V_{1, f^t}\left(s_1\right)-V_1^{\pi^t}\left(s_1\right)=\sum_{h=1}^H \mathbb{E}_{s_h, a_h \sim \pi^t}\left[Q_{h, f^t}\left(s_h, a_h\right)-r_h-V_{h+1, f^t}\left(s_{h+1}\right)\right]
$$

が成り立ちます．

## ステップ３

最後にリグレットをバウンドするために，ABCクラスのBellman Dominanceを使います．
$$
\kappa \cdot\left|\mathbb{E}_{s_h, a_h \sim \pi_f}\left[Q_{h, f}\left(s_h, a_h\right)-r\left(s_h, a_h\right)-V_{h+1, f}\left(s_{h+1}\right)\right]\right| \leq\left|G_{h, f^*}(f, f)\right| .
$$

これを使うと，
$$
\sum_{t=1}^T V_1^*\left(s_1\right)-V_1^{\pi^t}\left(s_1\right) \leq \frac{1}{\kappa} \sum_{t=1}^T \sum_{h=1}^H\left|G_{h, f^*}\left(f^t, f^t\right)\right|
$$
が成立します．

なので$\sum_{t=1}^T \sum_{h=1}^H\left|G_{h, f^*}\left(f^t, f^t\right)\right|$を上から抑えたいわけですね．
そのために，次の定理を証明します．

---

アルゴリズム１において，任意の$\rho > 0$と$\delta > 0$において，$\beta=c\left(\log \left(T H \mathcal{N}_{\mathcal{L}}(\rho) / \delta\right)+T \rho\right)$とします．また，定数$c$は十分大きいとします．
このとき，確率$1-\delta$以上で，任意の$(t, h) \in[T] \times[H]$について，

$$\sum_{i=1}^{t-1}\left(G_{h, f^*}\left(f^t, f^i\right)\right)^2 \leq \mathcal{O}(\beta)$$

が成立します．

---

証明はDominating Average Estimation Functionの性質と，$f^*$の実行可能解の証明を合体させたらできます．



### ステップ４

$\sum_{i=1}^t\left|G\left(f_i, g_i\right)\right|$をFunctional Eluder dimensionとステップ３をつかってバウンドします．

### ステップ５

後は全部合体させたら完成です．